<a href="https://colab.research.google.com/github/hainmit09/DataAnalysis/blob/main/DA_trong_l%C4%A9nh_v%E1%BB%B1c_b%C3%A1n_h%C3%A0ng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Nội dung học.
- DA trong marketing
- Các thuộc tính dữ liệu cần thiết của dữ liệu trong bài toán sales
- Các yêu cầu phần tích trong bài toán sale DA
- Các dạng báo cáo cần thiết trong bài toán sale DA
- Bài toán cụ thể

#2. DA trong lĩnh vực marketing
##2.1. Yêu cầu của DA trong marketing.

Trong lĩnh vực marketing, việc ứng dụng phân tích dữ liệu là rất quan trọng nhằm hiểu được tác động của tiếp thị, dự đoán xu hướng tiếp thị, hành vi người dùng, tối ưu hóa trải nghiệm người dùng, thúc đẩy doanh số bán hàng.

Có 2 yêu cầu chính của DA trong marketing
- Để đánh giá các nỗ lực tiếp thị của công ty đang hoạt động tốt như thế nào, hãy đo lường hiệu quả của hoạt động tiếp thị của bạn.
- Để xác định những gì công ty có thể làm khác đi để có được kết quả tốt hơn trên các kênh tiếp thị của mình.

##2.2. Tại sao DA trong marketing lại quan trọng.
Phân tích không phải là phần phụ, nó là cách tốt nhất để hiểu được hành trình của khách hàng, tìm ra điều gì hiệu quả trong các chiến dịch của bạn. Cụ thể
- Định lượng các yêu cầu
- Biến dữ liệu thành thông tin.
- So sánh và đối chiếu dữ liệu tiếp thị.
- Phân tích hiệu quả mục tiêu marketing


#3. Yêu cầu dữ liệu.
Để làm DA trong marketing cần có các dữ liệu sau:
##3.1. Dữ liệu khách hàng
- Nhóm tuổi
- Giới tính
- Trình độ học vấn
- Thu nhập hàng năm
- Tình trạng gia đình
- Vị trí địa lý

##3.2. Dữ liệu lịch sử bán hàng
- Mặt hàng.
- Thời gian bán.
- Địa điểm
- Giá bán.
- Giảm giá

##3.3. Thông tin môi trường.
- Thời tiết.
- Chỉ số thất nghiệp
- Giá các mặt hàng chi phối.

#4. Kết quả trả về.
#4.1. Dashboard
- Dạng ảnh.
- Dạng văn bản
- Dạng web
#4.2. Báo cáo
- Dạng file
- Dạng trực tuyến.

#5. Bài toán cụ thể


##5.1. Bài toán được giao
- Lập mô hình ảnh hưởng của giảm giá vào các tuần lễ
- Đưa ra các đề xuất, với ưu tiên càng cao khi tác động kinh doanh càng cao

##5.3 Đi xin dữ liệu
###5.3.1 Features
Chứa dữ liệu bổ sung liên quan đến hoạt động của cửa hàng, bộ phận và khu vực cho những ngày nhất định.

- Store - store number
- Date - ngày trong tuần
- Temperature - nhiệt độ trung bình trong khu vực
- Fuel_Price - chi phí nhiên liệu trong khu vực
- MarkDown1-5 - dữ liệu ẩn danh liên quan đến giảm giá khuyến mại. Dữ liệu MarkDown chỉ có sau tháng 11 năm 2011 và không phải lúc nào cũng có sẵn cho tất cả các cửa hàng. Bất kỳ giá trị nào bị thiếu đều được đánh dấu bằng NA
- CPI - chỉ số giá tiêu dùng
- Unemployment - tỷ lệ thất nghiệp
- IsHoliday - liệu tuần đó có phải là tuần lễ đặc biệt hay không

###5.3.2 Sales
Dữ liệu bán hàng lịch sử, bao gồm từ 2010-02-05 đến 2012-11-01.

- Store - store number
- Dept - department number
- Date -  ngày trong tuần
- Weekly_Sales -  doanh số bán hàng cho bộ phận nhất định trong cửa hàng nhất định
- IsHoliday - liệu tuần này có phải là tuần lễ đặc biệt hay không

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.style as style
from datetime import timedelta
import datetime as dt
import time
import os

In [ ]:
customer = pd.read_csv("/Customer.csv")
prod_cat= pd.read_csv("/prod_cat_info.csv")
transactions = pd.read_csv("Transactions.csv")

In [ ]:
#DATA PREPROCESSING
customer.isnull().sum()
"""Both Gender and city_code columns have null values"""

#To fix this, I applied ffill (fill forward) to the null cells
customers = customer.fillna({
    'Gender': customer['Gender'].ffill(),
    'city_code': customer['city_code'].ffill()
})

#Splitting transaction date into year, month and day of week

In [ ]:
transactions['tran_date'] = pd.to_datetime(transactions['tran_date'], errors='coerce')

In [ ]:
transactions.insert(loc=3, column='year', value= transactions.tran_date.dt.year)
transactions.insert(loc=4, column='month', value= transactions.tran_date.dt.month)
transactions.insert(loc=5, column='day', value=(transactions.tran_date.dt.weekday_name))

transactions.head()

In [ ]:
###### Transactions and Product Category datasets have no null cells.

df = pd.merge(left = customers, right = transactions, left_on = 'customer_Id', right_on = 'cust_id').drop('cust_id', axis =1)
#This joins the customers and transactions dataset on customer_Id and cust_id. The duplicate column (cust_id)
#is dropped.
df.duplicated().sum()
#There are 13 duplicate cells in the df dataframe. Next step: drop duplicates.

df.drop_duplicates(inplace = True)
df_new = pd.merge(df, prod_cat, left_on = ('prod_subcat_code', "prod_cat_code"), right_on = ('prod_sub_cat_code', "prod_cat_code")).drop('prod_sub_cat_code', axis =1)
df_new.shape
#Columns from the prod_cat dataset have been added to the df dataframe

df_new.describe()
#showing basic statistical details


customer_city=df_new[['city_code','customer_Id']]
customer_city.groupby(['city_code'])['customer_Id'].aggregate('count').reset_index().sort_values('customer_Id', ascending=False)

In [ ]:
rdf = df_new[df_new['Qty'] <= -1]
sdf = df_new[df_new['Qty'] >= 0]
"""To analyze negative and positive order values seperately, 2 new datasets were created."""

In [ ]:
#1. Returned Orders Per Store Category

In [ ]:
"""The e-shop accounted for the most returns in the returned orders dataset."""
orders = rdf.groupby(by=['Store_type'], as_index = False)['Qty'].count()
plt.figure(figsize=(6,4))
sns.set_style('whitegrid')
sns.barplot(x = "Store_type", y = 'Qty', data = orders,  palette= "magma")
plt.xlabel('Store Category')
plt.ylabel('Returned Orders')
plt.title('Total number of returned orders per store category')
plt.show()

In [ ]:
#2. Returned Orders Per Product Category
"""Books, Electronics and Home & Kitchen were the most returned product categories."""
category = rdf.groupby(by=['prod_cat'], as_index = False)['Qty'].count()
plt.figure(figsize=(8,4))
sns.set_style('whitegrid')
sns.barplot(x = "prod_cat", y = 'Qty', data = category,  palette= "inferno")
plt.xlabel('Product Category')
plt.ylabel('Returned Orders')
plt.title('Total number of returned orders per product category')
plt.show()

In [ ]:
#4. Returned Orders By City
"""Returns across cities were quite similar, with the highest recorded for City Code 8."""
city = rdf.groupby(by= ['city_code'], as_index = False)['Qty'].count()
plt.figure(figsize=(8,4))
sns.set_style('whitegrid')
sns.barplot(x = "city_code", y = 'Qty', data = city,  palette= "viridis")
plt.xlabel('City Code')
plt.ylabel('Returned Orders')
plt.title('Total number of returned orders per city')
plt.show()

In [ ]:
# 5. Returned Orders By Year
"""The highest product returns were recorded in 2012."""
order_year = rdf.groupby(by=['year'], as_index = False)['Qty'].count()
plt.figure(figsize=(6,5))
sns.barplot(x = "year", y = 'Qty', data = order_year,  palette= "plasma")
plt.xlabel('Year')
plt.ylabel('Returned Orders')
plt.title('Returned Orders Per Year')
plt.show()

"""Chart below shows revenue loss."""
sales = rdf.groupby(by=['year'], as_index = False)['total_amt'].sum()
plt.figure(figsize=(6,5))
sns.barplot(x = "year", y = 'total_amt', data = sales,  palette = 'plasma')
plt.xlabel('Year')
plt.ylabel('Returned Orders')
plt.title('Revenue Lost Due To Returns')
plt.show()

In [ ]:
Successful Orders Dataset

1. Total Orders Per Store Category

"""The most purchases were made through the e-shop."""
orders1 = sdf.groupby(by=['Store_type'], as_index = False)['Qty'].count()
plt.figure(figsize=(6,4))
sns.set_style('whitegrid')
sns.barplot(x = "Store_type", y = 'Qty', data = orders1,  palette= "inferno")
plt.xlabel('Store Category')
plt.ylabel('Total Orders')
plt.title('Total successful orders per store category')
plt.show()



In [ ]:
2. Total Orders Per Product Category
"""Books, Electronics and Home & Kitchen were the most purchased product categories."""
category1 = sdf.groupby(by=['prod_cat'], as_index = False)['Qty'].count()
plt.figure(figsize=(8,4))
sns.set_style('whitegrid')
sns.barplot(x = "prod_cat", y = 'Qty', data = category1,  palette= "plasma")
plt.xlabel('Product Category')
plt.ylabel('Total Orders')
plt.title('Total successful orders per product category')
plt.show()

In [ ]:
3. Total Successful Orders Per City
location1 = sdf.groupby(by= ['city_code'], as_index = False)['Qty'].count()
plt.figure(figsize=(8,4))
sns.set_style('whitegrid')
sns.barplot(x = "city_code", y = 'Qty', data = location1,  palette= "viridis")
plt.xlabel('City Code')
plt.ylabel('Successful Orders')
plt.title('Orders placed per location')
plt.show()

In [ ]:
5. Total Orders Per Year
"""Highest sales occured in the years 2012 and 2013."""
order_year1 = sdf.groupby(by=['year'], as_index = False)['Qty'].count()
plt.figure(figsize=(6,5))
sns.barplot(x = "year", y = 'Qty', data = order_year1,  palette= "plasma")
plt.xlabel('Year')
plt.ylabel('Total Orders')
plt.title('Order Quantity Per Year')
plt.show()

"""The most successful sales occured in 2012 and 2013"""
sales1 = sdf.groupby(by=['year'], as_index = False)['total_amt'].sum()
plt.figure(figsize=(6,5))
sns.barplot(x = "year", y = 'total_amt', data = sales1,  palette= "plasma")
plt.xlabel('Year')
plt.ylabel('Total Orders')
plt.title('Revenue Generated')
plt.show()

In [ ]:
6. Revenue Per Category
"""Products in the Women, Mens and Kids categories sold better than other categories."""
subcategory = sdf.groupby(by=['prod_subcat'], as_index = False)['total_amt'].sum()
plt.figure(figsize=(8,6))
sns.set_style('whitegrid')
sns.barplot(x = "total_amt", y = 'prod_subcat', data = subcategory, palette= "inferno")
plt.xlabel('Amount Spent')
plt.ylabel('Product Subcategories')
plt.title('Amount Spent Per Subcategories')
plt.show()

In [ ]:
7. Purchase By Gender
"""Purchases by men accounted for the highest percentage across all product categories except footwear and bags."""
plt.figure(figsize=(8,4))
sns.set_style('whitegrid')
sns.countplot(x = 'prod_cat', hue = "Gender", data = sdf, palette= "inferno")
plt.xlabel('Amount Spent')
plt.ylabel('Product Categories')
plt.title('Purchase By Gender')
plt.show()

"""Pivot chart representation"""
product_by_gender = sdf.groupby(["Gender","prod_cat"])[["Qty"]].sum().reset_index()
product_by_gender.pivot(index="Gender",columns="prod_cat",values="Qty")

In [ ]:
"""To analyze data based on customer age, a new age column is created."""
now = pd.Timestamp('now')
sdf['DOB'] = pd.to_datetime(sdf['DOB'], errors = 'coerce')    #1
sdf['DOB'] = sdf['DOB'].where(sdf['DOB'] < now, sdf['DOB'] -  np.timedelta64(100, 'Y'))   # 2
sdf['AGE'] = (now - sdf['DOB']).astype('<m8[Y]').round()

#check for max and min age
sdf['AGE'].max()
sdf['AGE'].min()

"""The cut() method is used to bin age values into discrete intervals."""
sdf['age_category'] = pd.cut(x = sdf['AGE'], bins = [24, 30, 39, 49], labels=['24-29','30-39','40-50'],include_lowest=True)

In [ ]:
sdf['age_category'].value_counts()

In [ ]:
7. Purchase By Age Category
"""Customers aged betweeen 40-50 purchased the most products and 24-30 customers purchased the least"""
plt.figure(figsize=(8,6))
sns.countplot(x = 'prod_cat', hue = 'age_category', data = sdf, palette= "inferno")

"""Pivot chart representation"""
spend_per_category = sdf.groupby(['age_category','prod_cat'])['total_amt'].sum().reset_index()
spend_per_category.pivot(index = "age_category", columns = "prod_cat", values = 'total_amt').round(0)

In [ ]:
7. Quantity Sold By Month
"""Showing category sales by month."""
plt.figure(figsize=(12,6))
sns.countplot(x = 'prod_cat', hue = 'month', data = sdf, palette= "plasma")

sale_by_month = sdf.groupby(['month','prod_cat'])['Qty'].count().reset_index()
sale_by_month.pivot(index = "month", columns = "prod_cat", values = 'Qty').round(0)

sdf['month'].value_counts()

In [ ]:
RFM Score Calculation
"""When it comes to finding out who your best customers are, the old RFM matrix principle is the best. RFM stands for Recency, Frequency and Monetary. It is a customer segmentation technique that uses past purchase behavior to divide customers into groups.
RFM Score Calculations
RECENCY (R): Days since last purchase
FREQUENCY (F): Total number of purchases
MONETARY VALUE (M): Total money this customer spent"""

df_new['tran_date'].min()

df_new['tran_date'].max()

NOW = dt.datetime(2014,12,3)

rfmTable = sdf.groupby('customer_Id').agg({'tran_date': lambda x: (NOW - x.max()).days, 'transaction_id': lambda x: len(x), 'total_amt': lambda x: x.sum()})

rfmTable['tran_date'] = rfmTable['tran_date'].astype(int)

rfmTable.rename(columns={'tran_date': 'recency', 
                         'transaction_id': 'frequency', 
                         'total_amt': 'monetary_value'}, inplace=True)

rfmTable.head()


In [ ]:
sort_by_monetary_value = rfmTable.sort_values('monetary_value',ascending=False)

"""Customers with the highest purchases:"""
print(sort_by_monetary_value.head(n=10))

In [ ]:
"""Getting rows for the most valuable customer."""
most_valued_customer = sdf.loc[sdf['customer_Id'] == 271834]

In [ ]:
print(most_valued_customer)